# 시군구별 법정동, 행정구역코드 매칭

같은 시, 같은 동의 법정동코드와 행정구역 코드를 매칭한 `dong_code_matching.csv` 파일을 생성하는 코드입니다.

### 1. 배경

프로젝트를 진행하던 중,  
공공데이터에서 `법정동` 기준으로 제공되는 매물 거래내역 데이터와  
통계청에서 `행정구역코드` 기준으로 제공되는 인구통계 데이터를  
매칭 시켜야 하는 문제가 생겼습니다.

따라서 통계청 및 공공데이터에서 각각 시군구별 코드자료를 받은 다음
매칭시키는 파이썬 코드를 작성했습니다.

예를들어, **서울특별시 종로구**의 경우 법정동코드는 `11110(00000)` 이지만,  행정구역코드는 `11010` 입니다.

### 2. 코드


In [4]:
import pandas as pd

#법정동코드
dong_code = pd.read_csv('./국토교통부_전국 법정동_20240802.csv', encoding="cp949")
dong_code["법정동코드"] = dong_code["법정동코드"].astype(str)
dong_code = dong_code[dong_code["삭제일자"].isna()]
dong_code = dong_code[["법정동코드","시도명","시군구명"]]
dong_code.head()

,법정동코드,시도명,시군구명
0,1100000000,서울특별시,NaN
1,1111000000,서울특별시,종로구
2,1111010100,서울특별시,종로구
3,1111010200,서울특별시,종로구
4,1111010300,서울특별시,종로구


In [5]:
# 1. '법정동코드' 칼럼의 앞 5자리만 잘라서 다시 넣기
dong_code["법정동코드"] = dong_code["법정동코드"].str[:5]

# 2. '시군구명'이 NaN인 행 제거
dong_code = dong_code.dropna(subset=["시군구명"])

# 3. '법정동코드'가 동일한 행은 첫 번째 데이터만 남기기
dong_code = dong_code.drop_duplicates(subset=["법정동코드"], keep="first")

# 4. 시도명 시군구명 합치기
dong_code["시도명"] = dong_code["시도명"]+"_"+dong_code["시군구명"]
dong_code = dong_code[["법정동코드","시도명"]]
# 결과 출력
# pd.set_option('display.max_rows', None) # 모든 행을 보고 싶은 경우
dong_code.head()




,법정동코드,시도명
1,11110,서울특별시_종로구
94,11140,서울특별시_중구
179,11170,서울특별시_용산구
229,11200,서울특별시_성동구
332,11215,서울특별시_광진구


In [14]:
def tothree(data):
  if len(data)== 2:
    return "0"+data
  else :
    return data

def remove_space(data):
  return "".join(data.split(" "))


#행정구역코드
adm_code = pd.read_csv('./adm_code.csv',encoding="cp949")

adm_code['행정구역코드'] = adm_code['시도코드'].astype(str) + adm_code['시군구코드'].astype(str).apply(tothree)

# 1. '행정구역코드'가 동일한 행은 첫 번째 데이터만 남기기
adm_code = adm_code.drop_duplicates(subset=["행정구역코드"], keep="first")
adm_code

# 2. 시군구명칭 공백 없애기
adm_code["시군구명칭"] = adm_code["시군구명칭"].apply(remove_space)
# 3. 시도명 통일하기
adm_code["시도명"] = adm_code["시도명칭"]+"_"+adm_code["시군구명칭"]
adm_code = adm_code[["행정구역코드","시도명"]]

adm_code.head()


,행정구역코드,시도명
0,11010,서울특별시_종로구
17,11020,서울특별시_중구
32,11030,서울특별시_용산구
48,11040,서울특별시_성동구
65,11050,서울특별시_광진구


In [16]:
print(f'행정구역코드 개수는 {len(adm_code)} 이고, 법정동 코드 개수는 {len(dong_code)} 이다.')

행정구역코드 개수는 250 이고, 법정동 코드 개수는 264 이다.


In [18]:
# 데이터프레임 조인 (right join)
merged_df = pd.merge(dong_code, adm_code, on="시도명", how="right")
merged_df["adm_cd"] = merged_df["행정구역코드"]
merged_df["city_name"] =merged_df["시도명"]
merged_df["dong_code"] =merged_df["법정동코드"]

merged_df = merged_df[["adm_cd","dong_code","city_name"]]
merged_df.head()



,adm_cd,dong_code,city_name
0,11010,11110,서울특별시_종로구
1,11020,11140,서울특별시_중구
2,11030,11170,서울특별시_용산구
3,11040,11200,서울특별시_성동구
4,11050,11215,서울특별시_광진구


In [19]:
merged_df.to_csv("dong_code_matching.csv",encoding="utf-8-sig")